In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../orig-power-soton-page-flow.csv')

In [3]:
df

,user number,session number,session start time,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 159,Unnamed: 160,Unnamed: 161,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168
0,24015,1,42594.46225,homepage,5,registration,27,registrationfinish#1,16,usingpower,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24024,1,42597.50486,homepage,7,registration,85,registrationfinish,7,usingpower,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24024,2,42604.55251,homepage,40,login,9,login2,4,welcomeback3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24024,3,42611.65603,homepage,5,login,8,login2,7,welcomeback4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24024,4,42618.50609,homepage,61,login,35,login2,7,welcomeback5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,24024,5,42625.59153,homepage,5,login,7,login2,9,welcomeback6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,24024,6,42632.63392,homepage,4,login,9,login2,10,welcomeback7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,24024,7,42633.81534,homepage,5,login,9,login2,10,s2weight,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,24067,1,42605.39964,homepage,146,registration,217,registrationfinish,28,usingpower,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,24067,2,42612.48280,homepage,15,login,25,login2,6,welcomeback3#1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_transformed = pd.DataFrame(
    columns = ['user_number', 'session_number', 'session_start_time', 'source', 'source_duration', 'target']
)
column_names = df.columns.values.tolist()
new_row_index = 0
for index, row in df.iterrows():
    for column_name in column_names:
        if 'Unnamed' in column_name and not int(column_name[9:]) % 2 == 0:
            column_number = int(column_name[9:])
            if column_number <= 167:
                df_transformed.set_value(new_row_index, 'source', row[column_name])
                df_transformed.set_value(new_row_index, 'source_duration', row[column_name[:9] + str(column_number + 1)])
            if column_number <= 166:
                df_transformed.set_value(new_row_index, 'target', row[column_name[:9] + str(column_number + 2)])
            
            df_transformed.set_value(new_row_index, 'user_number', row['user number'])
            df_transformed.set_value(new_row_index, 'session_number', row['session number'])
            df_transformed.set_value(new_row_index, 'session_start_time', row['session start time'])
            
            new_row_index += 1
df_transformed

,user_number,session_number,session_start_time,source,source_duration,target
0,24015,1,42594.5,homepage,5,registration
1,24015,1,42594.5,registration,27,registrationfinish#1
2,24015,1,42594.5,registrationfinish#1,16,usingpower
3,24015,1,42594.5,usingpower,11,registrationfinish#2
4,24015,1,42594.5,registrationfinish#2,0,q_followup_8wk
5,24015,1,42594.5,q_followup_8wk,4,q_weight
6,24015,1,42594.5,q_weight,11,q_therapist
7,24015,1,42594.5,q_therapist,10,q_websat
8,24015,1,42594.5,q_websat,5,q_patientenable_1
9,24015,1,42594.5,q_patientenable_1,6,q_complete_8wk


In [48]:
df.dtypes

user number             int64
session number          int64
session start time    float64
Unnamed: 3             object
Unnamed: 4              int64
Unnamed: 5             object
Unnamed: 6              int64
Unnamed: 7             object
Unnamed: 8             object
Unnamed: 9             object
Unnamed: 10            object
Unnamed: 11            object
Unnamed: 12           float64
Unnamed: 13            object
Unnamed: 14            object
Unnamed: 15            object
Unnamed: 16           float64
Unnamed: 17            object
Unnamed: 18            object
Unnamed: 19            object
Unnamed: 20            object
Unnamed: 21            object
Unnamed: 22            object
Unnamed: 23            object
Unnamed: 24            object
Unnamed: 25            object
Unnamed: 26           float64
Unnamed: 27            object
Unnamed: 28            object
Unnamed: 29            object
                       ...   
Unnamed: 139           object
Unnamed: 140           object
Unnamed: 1